In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import (roc_auc_score, accuracy_score, average_precision_score,
                             precision_score, recall_score, roc_curve, precision_recall_curve)
import matplotlib.pyplot as plt
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


class ImprovedDataset(Dataset):
    """Dataset class for immunogenicity prediction"""
    
    def __init__(self, dataframe, aaindex_path="./data/aaindex1_pca.csv"):
        self.data = dataframe.reset_index(drop=True)
        self.aaindex = pd.read_csv(aaindex_path)
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        pseudosequence = row['pseudosequence']
        peptide = row['Peptide']
        label = row['Label']
        tap = row['tap_prediction_score']
        rank = row['%Rank_EL']
        
        pseudo_onehot = self._onehot_encoding(pseudosequence, 34)
        peptide_onehot = self._onehot_encoding(peptide, 11)
        pseudo_aa = self._get_AA_features(pseudosequence, 34)
        peptide_aa = self._get_AA_features(peptide, 11)
        
        pseudo_features = torch.cat([pseudo_onehot, pseudo_aa], dim=1)  # [34, 42]
        peptide_features = torch.cat([peptide_onehot, peptide_aa], dim=1)  # [11, 42]
        
        global_features = torch.tensor([tap, rank], dtype=torch.float32)
        
        return (pseudo_features.float(), 
                peptide_features.float(), 
                global_features, 
                torch.tensor(label, dtype=torch.long))
    
    def _onehot_encoding(self, sequence, maxlen):
        sequence = sequence.upper()[:maxlen]
        enc_seq = torch.zeros((maxlen, 20), dtype=torch.float32)
        
        for i, aa in enumerate(sequence):
            if aa in self.amino_acids:
                enc_seq[i, self.amino_acids.index(aa)] = 1
        
        return enc_seq
    
    def _get_AA_features(self, sequence, maxlen):
        sequence = sequence.ljust(maxlen, 'X')[:maxlen]
        
        all_node_feats = []
        for index, aa in enumerate(sequence):
            node_feats = self.aaindex[aa].to_list()
            anchar = [0, maxlen]
            seq_onehot = [0, 0]
            seq_onehot[sum([index >= i for i in anchar]) - 1] = 1
            node_feats.extend(seq_onehot)
            all_node_feats.append(node_feats)
        
        return torch.tensor(all_node_feats, dtype=torch.float32)


# ==================== Model 1: CNN (Original) ====================
class ImprovedCNN(nn.Module):
    """CNN-based model for local motif detection"""
    
    def __init__(self):
        super(ImprovedCNN, self).__init__()
        
        self.pseudo_branch = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 32, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1), stride=2)
        )
        
        self.peptide_branch = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 1), stride=2)
        )
        
        self.conv_output_dim = self._get_conv_output()
        
        self.classifier = nn.Sequential(
            nn.Linear(self.conv_output_dim + 2, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )
    
    def _get_conv_output(self):
        with torch.no_grad():
            x1 = torch.zeros(1, 1, 34, 42)
            x2 = torch.zeros(1, 1, 11, 42)
            x1 = self.pseudo_branch(x1)
            x2 = self.peptide_branch(x2)
            x1_flat = x1.view(1, -1).size(1)
            x2_flat = x2.view(1, -1).size(1)
            return x1_flat + x2_flat
    
    def forward(self, pseudo, peptide, global_feat):
        x1 = pseudo.unsqueeze(1)
        x2 = peptide.unsqueeze(1)
        
        x1 = self.pseudo_branch(x1)
        x2 = self.peptide_branch(x2)
        
        x1 = x1.view(x1.size(0), -1)
        x2 = x2.view(x2.size(0), -1)
        
        x = torch.cat([x1, x2, global_feat], dim=1)
        x = self.classifier(x)
        return x


# ==================== Model 2: BiLSTM ====================
class BiLSTMModel(nn.Module):
    """BiLSTM-based model for sequential pattern learning"""
    
    def __init__(self, hidden_dim=128, num_layers=2):
        super(BiLSTMModel, self).__init__()
        
        # LSTM for pseudo sequence
        self.pseudo_lstm = nn.LSTM(
            input_size=42,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.3
        )
        
        # LSTM for peptide sequence
        self.peptide_lstm = nn.LSTM(
            input_size=42,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=0.3
        )
        
        # Classifier
        lstm_output_dim = hidden_dim * 2 * 2  # bidirectional * 2 sequences
        self.classifier = nn.Sequential(
            nn.Linear(lstm_output_dim + 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )
    
    def forward(self, pseudo, peptide, global_feat):
        # Process pseudo sequence
        pseudo_out, (pseudo_h, _) = self.pseudo_lstm(pseudo)
        # Concatenate forward and backward hidden states from last layer
        pseudo_feat = torch.cat([pseudo_h[-2], pseudo_h[-1]], dim=1)
        
        # Process peptide sequence
        peptide_out, (peptide_h, _) = self.peptide_lstm(peptide)
        peptide_feat = torch.cat([peptide_h[-2], peptide_h[-1]], dim=1)
        
        # Concatenate all features
        x = torch.cat([pseudo_feat, peptide_feat, global_feat], dim=1)
        x = self.classifier(x)
        return x


# ==================== Model 3: Transformer ====================
class TransformerModel(nn.Module):
    """Transformer-based model with self-attention mechanism"""
    
    def __init__(self, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        
        # Input projection
        self.pseudo_proj = nn.Linear(42, d_model)
        self.peptide_proj = nn.Linear(42, d_model)
        
        # Positional encoding
        self.pseudo_pos = nn.Parameter(torch.randn(1, 34, d_model))
        self.peptide_pos = nn.Parameter(torch.randn(1, 11, d_model))
        
        # Transformer encoders
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=512,
            dropout=0.3,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(d_model * 2 + 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )
    
    def forward(self, pseudo, peptide, global_feat):
        # Project and add positional encoding
        pseudo_x = self.pseudo_proj(pseudo) + self.pseudo_pos
        peptide_x = self.peptide_proj(peptide) + self.peptide_pos
        
        # Apply transformer
        pseudo_out = self.transformer(pseudo_x)
        peptide_out = self.transformer(peptide_x)
        
        # Global average pooling
        pseudo_feat = pseudo_out.mean(dim=1)
        peptide_feat = peptide_out.mean(dim=1)
        
        # Concatenate and classify
        x = torch.cat([pseudo_feat, peptide_feat, global_feat], dim=1)
        x = self.classifier(x)
        return x


# ==================== Model 4: Hybrid CNN-LSTM ====================
class HybridCNNLSTM(nn.Module):
    """Hybrid model combining CNN for local features and LSTM for sequential patterns"""
    
    def __init__(self):
        super(HybridCNNLSTM, self).__init__()
        
        # CNN layers for local motif extraction
        self.pseudo_conv = nn.Sequential(
            nn.Conv1d(42, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        self.peptide_conv = nn.Sequential(
            nn.Conv1d(42, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )
        
        # LSTM layers for sequential patterns
        self.pseudo_lstm = nn.LSTM(64, 64, batch_first=True, bidirectional=True)
        self.peptide_lstm = nn.LSTM(64, 64, batch_first=True, bidirectional=True)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(64 * 4 + 2, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 2)
        )
    
    def forward(self, pseudo, peptide, global_feat):
        # CNN processing (transpose for Conv1d: batch, channels, length)
        pseudo_x = self.pseudo_conv(pseudo.transpose(1, 2))
        peptide_x = self.peptide_conv(peptide.transpose(1, 2))
        
        # Transpose back for LSTM
        pseudo_x = pseudo_x.transpose(1, 2)
        peptide_x = peptide_x.transpose(1, 2)
        
        # LSTM processing
        _, (pseudo_h, _) = self.pseudo_lstm(pseudo_x)
        _, (peptide_h, _) = self.peptide_lstm(peptide_x)
        
        # Concatenate bidirectional hidden states
        pseudo_feat = torch.cat([pseudo_h[0], pseudo_h[1]], dim=1)
        peptide_feat = torch.cat([peptide_h[0], peptide_h[1]], dim=1)
        
        # Concatenate all features
        x = torch.cat([pseudo_feat, peptide_feat, global_feat], dim=1)
        x = self.classifier(x)
        return x


# ==================== Training and Evaluation Functions ====================
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    all_preds, all_labels = [], []
    
    for pseudo, peptide, global_feat, labels in tqdm(loader, desc="Training", leave=False):
        pseudo = pseudo.to(device)
        peptide = peptide.to(device)
        global_feat = global_feat.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(pseudo, peptide, global_feat)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    return avg_loss, accuracy


def evaluate_model(model, criterion, dataloader, device):
    model.eval()
    all_labels = []
    all_predictions = []
    all_probs = []
    running_loss = 0.0
    
    with torch.no_grad():
        for pseudo, peptide, global_feat, labels in tqdm(dataloader, desc="Evaluation", leave=False):
            pseudo = pseudo.to(device)
            peptide = peptide.to(device)
            global_feat = global_feat.to(device)
            labels = labels.to(device)
            
            outputs = model(pseudo, peptide, global_feat)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            probs = F.softmax(outputs, dim=1)[:, 1]
            predictions = torch.argmax(outputs, dim=1)
            
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    avg_loss = running_loss / len(dataloader)
    
    precision = precision_score(all_labels, all_predictions, zero_division=0)
    recall = recall_score(all_labels, all_predictions, zero_division=0)
    auc = roc_auc_score(all_labels, all_probs)
    aupr = average_precision_score(all_labels, all_probs)
    acc = accuracy_score(all_labels, all_predictions)
    
    return {
        'loss': avg_loss,
        'auc': auc,
        'aupr': aupr,
        'acc': acc,
        'precision': precision,
        'recall': recall
    }


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def train_and_evaluate(model_class, model_name, train_loader, val_loader, 
                       num_epochs=30, lr=0.0001, device=device):
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}\n")
    
    model = model_class().to(device)
    print(f"Model parameters: {count_parameters(model):,}")
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
    
    best_auc = 0
    best_metrics = None
    training_time = 0
    
    for epoch in range(num_epochs):
        start_time = time.time()
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        val_metrics = evaluate_model(model, criterion, val_loader, device)
        
        epoch_time = time.time() - start_time
        training_time += epoch_time
        
        print(f"Epoch {epoch+1}/{num_epochs} ({epoch_time:.1f}s)")
        print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"  Val   - Loss: {val_metrics['loss']:.4f}, Acc: {val_metrics['acc']:.4f}, "
              f"AUC: {val_metrics['auc']:.4f}, AUPR: {val_metrics['aupr']:.4f}")
        
        if val_metrics['auc'] > best_auc:
            best_auc = val_metrics['auc']
            best_metrics = val_metrics.copy()
            torch.save(model.state_dict(), f'best_{model_name}.pth')
            print(f"  [*] Best model saved!")
        print()
    
    best_metrics['training_time'] = training_time
    best_metrics['parameters'] = count_parameters(model)
    
    return model, best_metrics


# ==================== Main Comparison ====================
if __name__ == "__main__":
    print("Loading data...")
    data = pd.read_csv("data/output_binding_tap.csv")
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)
    
    train_dataset = ImprovedDataset(train_data)
    val_dataset = ImprovedDataset(val_data)
    
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    
    print(f"Train samples: {len(train_dataset)}")
    print(f"Val samples: {len(val_dataset)}")
    
    # Define models to compare
    models = [
        (ImprovedCNN, "CNN"),
        (BiLSTMModel, "BiLSTM"),
        (TransformerModel, "Transformer"),
        (HybridCNNLSTM, "CNN-LSTM")
    ]
    
    results = {}
    
    # Train each model
    for model_class, model_name in models:
        try:
            _, metrics = train_and_evaluate(
                model_class, 
                model_name,
                train_loader, 
                val_loader,
                num_epochs=20,  # 减少到10轮用于快速测试
                lr=0.0001
            )
            results[model_name] = metrics
        except Exception as e:
            print(f"Error training {model_name}: {e}")
            continue
    
    # Print comparison table
    print("\n" + "="*80)
    print("COMPARATIVE RESULTS")
    print("="*80)
    print(f"{'Model':<15} {'AUC':<8} {'AUPR':<8} {'Acc':<8} {'Prec':<8} {'Recall':<8} {'Params':<10} {'Time(s)':<10}")
    print("-"*80)
    
    for model_name, metrics in results.items():
        print(f"{model_name:<15} "
              f"{metrics['auc']:<8.4f} "
              f"{metrics['aupr']:<8.4f} "
              f"{metrics['acc']:<8.4f} "
              f"{metrics['precision']:<8.4f} "
              f"{metrics['recall']:<8.4f} "
              f"{metrics['parameters']:<10,} "
              f"{metrics['training_time']:<10.1f}")
    
    # Save results
    results_df = pd.DataFrame(results).T
    results_df.to_csv('model_comparison_results.csv')
    print("\nResults saved to 'model_comparison_results.csv'")


Using device: cpu
Loading data...
Train samples: 6267
Val samples: 1567

Training CNN

Model parameters: 14,594,498


Epoch 1/20 (15.0s)
  Train - Loss: 0.5951, Acc: 0.6675
  Val   - Loss: 0.5015, Acc: 0.7371, AUC: 0.8276, AUPR: 0.7370
  [*] Best model saved!



Epoch 2/20 (14.5s)
  Train - Loss: 0.5022, Acc: 0.7421
  Val   - Loss: 0.4680, Acc: 0.7913, AUC: 0.8592, AUPR: 0.7825
  [*] Best model saved!



Epoch 3/20 (14.6s)
  Train - Loss: 0.4874, Acc: 0.7581
  Val   - Loss: 0.4571, Acc: 0.7671, AUC: 0.8619, AUPR: 0.7886
  [*] Best model saved!



Epoch 4/20 (14.4s)
  Train - Loss: 0.4816, Acc: 0.7591
  Val   - Loss: 0.4500, Acc: 0.7735, AUC: 0.8683, AUPR: 0.7989
  [*] Best model saved!



...

Epoch 16/20 (27.5s)
  Train - Loss: 0.4526, Acc: 0.7796
  Val   - Loss: 0.4309, Acc: 0.7894, AUC: 0.8720, AUPR: 0.8028
  [*] Best model saved!



Epoch 17/20 (28.8s)
  Train - Loss: 0.4468, Acc: 0.7803
  Val   - Loss: 0.4249, Acc: 0.7875, AUC: 0.8742, AUPR: 0.8046
  [*] Best model saved!



Epoch 18/20 (28.6s)
  Train - Loss: 0.4465, Acc: 0.7824
  Val   - Loss: 0.4223, Acc: 0.7926, AUC: 0.8754, AUPR: 0.8065
  [*] Best model saved!



Epoch 19/20 (12.4s)
  Train - Loss: 0.4403, Acc: 0.7835
  Val   - Loss: 0.4196, Acc: 0.7951, AUC: 0.8766, AUPR: 0.8088
  [*] Best model saved!



Epoch 20/20 (11.4s)
  Train - Loss: 0.4395, Acc: 0.7852
  Val   - Loss: 0.4159, Acc: 0.7945, AUC: 0.8780, AUPR: 0.8110
  [*] Best model saved!


COMPARATIVE RESULTS
Model           AUC      AUPR     Acc      Prec     Recall   Params     Time(s)   
--------------------------------------------------------------------------------
CNN             0.8919   0.8382   0.8175   0.7757   0.7143   14,594,498 329.5     
BiLSTM          0.8825   0.8194   0.8034   0.7205   0.7676   1,472,386  292.8     
Transformer     0.8768   0.7965   0.8009   0.7663   0.6661   810,114    570.0     
CNN-LSTM        0.8780   0.8110   0.7945   0.7342   0.6988   360,642    265.2     

Results saved to 'model_comparison_results.csv'
